In [10]:
import requests
import re
import pprint
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
from sklearn.naive_bayes import MultinomialNB

## Create URL list per artist

In [11]:
#artist page list
artists = ['J.Cole',
           'Ed-Sheeran', 
           'Radiohead',
           'Taylor-Swift']
urls = ['https://www.lyrics.com/artist/J.-Cole/1557103',
        'https://www.lyrics.com/artist/Ed-Sheeran/2342870',
        'https://www.lyrics.com/artist/Radiohead/41092',
        'https://www.lyrics.com/artist/Taylor-Swift/816977']
artist_df= pd.DataFrame(artists, index=urls)
artist_df #create df to get artist name. Is there any better  way?

,0
https://www.lyrics.com/artist/J.-Cole/1557103,J.Cole
https://www.lyrics.com/artist/Ed-Sheeran/2342870,Ed-Sheeran
https://www.lyrics.com/artist/Radiohead/41092,Radiohead
https://www.lyrics.com/artist/Taylor-Swift/816977,Taylor-Swift


In [12]:
artist_df.loc['https://www.lyrics.com/artist/J.-Cole/1557103', 0]

'J.Cole'

In [13]:
complete_url_list = []
# labels.clear()
labels = []
for url in urls:
    html = requests.get(url).text
    #pprint.pprint(html)
    song_list = re.findall(pattern='/lyric/[^"]+', string=html) # Get the song links
    #pprint.pprint(song_list)
    prefix_url = 'https://www.lyrics.com/' #Get the complete urls

    for hyper_link in song_list[0:10]: #limit the number of song of each artist
        complete_url = prefix_url + hyper_link
        complete_url_list.append(complete_url)
        labels.append(artist_df.loc[url, 0]) #get artist name from artist_df. Is there any other better way?
#print(complete_url_list, labels)

In [14]:
url_df= pd.DataFrame(labels, index=complete_url_list)
url_df #create df to get artist name. Is there any better  way?

,0
https://www.lyrics.com//lyric/36437994/J.+Cole/Jodeci+Freestyle,J.Cole
https://www.lyrics.com//lyric/36590346/J.+Cole/Family+and+Loyalty,J.Cole
https://www.lyrics.com//lyric/36021592/J.+Cole/How+Did+I+Get+Here,J.Cole
https://www.lyrics.com//lyric/36086297/J.+Cole/How+Did+I+Get+Here,J.Cole
https://www.lyrics.com//lyric/36385934/J.+Cole/The+London,J.Cole
https://www.lyrics.com//lyric/36551272/J.+Cole/Prove+It,J.Cole
https://www.lyrics.com//lyric/35952125/J.+Cole/Middle+Child,J.Cole
https://www.lyrics.com//lyric/36389078/J.+Cole/The+London,J.Cole
https://www.lyrics.com//lyric/36185620/J.+Cole/Middle+Child,J.Cole
https://www.lyrics.com//lyric/36231744/J.+Cole/Purple+Emoji,J.Cole


## Create corpus

In [15]:
#corpus.clear()
corpus = []
#labels_2.clear()
labels_2 =[]
#labels.clear()
for url in complete_url_list:
    response = requests.get(url, allow_redirects=False)
    if response.status_code == 302: #error page(redirect)
        continue
    lyrics = response.text
    
    lyrics_soup = BeautifulSoup(lyrics, 'html.parser')
    title_text = lyrics_soup.body.find(class_ = 'lyric-title').text
    lyrics_text = lyrics_soup.body.find('pre', attrs={'class': 'lyric-body'}).text
    lyrics_text = lyrics_text.replace('\r\n',' ')

    #titles.append(title_text)
    labels_2.append(url_df.loc[url, 0]) #get artist name from url_df
    corpus.append(lyrics_text)

In [16]:
nltk.download('stopwords')
STOPWORDS = stopwords.words('english')
print(STOPWORDS)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shoheisuzuki/miniforge3/lib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Use TfVectorizer(combination of CounterVectorizer and TfIdfTransformer)

In [17]:
vectorizer = TfidfVectorizer(max_df=0.8, stop_words=STOPWORDS) #instanciation
vectors = vectorizer.fit_transform(corpus)

#for us to see vectorized labeled data
pd.DataFrame(vectors.todense(), columns=vectorizer.get_feature_names(), index=labels_2)

/Users/shoheisuzuki/miniforge3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,06,100,12,16th,21,26,50,89,94,96,...,wrote,ya,yard,yeah,year,years,yes,yet,young,youth
J.Cole,0.042101,0.000000,0.042101,0.000000,0.000000,0.042101,0.037855,0.000000,0.042101,0.042101,...,0.000000,0.000000,0.000000,0.017478,0.000000,0.000000,0.000000,0.037855,0.030596,0.000000
J.Cole,0.000000,0.020118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.016703,0.000000,0.015533,0.000000,0.000000,0.000000,0.018089
J.Cole,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.069625,0.000000,0.000000,0.081493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
J.Cole,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.069625,0.000000,0.000000,0.081493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
J.Cole,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039836,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.220710,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
J.Cole,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.031809,0.000000,0.038436,0.143606,0.034559,0.000000,0.000000,0.000000,0.000000,0.000000
J.Cole,0.000000,0.000000,0.000000,0.000000,0.040747,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.040747,0.000000,0.000000,0.000000,0.098801,0.000000
J.Cole,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039836,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.220710,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
J.Cole,0.000000,0.000000,0.000000,0.000000,0.040861,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.099077,0.000000
J.Cole,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045622,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.105320,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
#pd.set_option("display.max_columns", 1000)

## Classification NB model

In [19]:
model = MultinomialNB()

In [20]:
# fit the model with the vectorized data
model.fit(vectors, labels_2)  

MultinomialNB()

In [21]:
# multinomialNB usually tends to overfit
model.score(vectors, labels_2)

1.0

In [22]:
test_lyrics = ['your kingdom keys', 'I love music', 'Im devoted to making sure that shit goes unnoticed']
test_vectors = vectorizer.transform(test_lyrics)

In [23]:
test_vectors.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
model.predict(test_vectors)

array(['Taylor-Swift', 'Ed-Sheeran', 'J.Cole'], dtype='<U12')

In [25]:
model.predict_proba(test_vectors)

array([[0.24828943, 0.24495546, 0.25043701, 0.2563181 ],
       [0.28886234, 0.23124244, 0.23418396, 0.24571127],
       [0.24047771, 0.28034387, 0.25767991, 0.22149851]])

In [26]:
#next
#preprocessing of corpus
#test train split
#remove duplication by dictionary